In [4]:
import numpy as np
import warnings

warnings.filterwarnings("ignore")

In [5]:
def MOL_American_Put(Stock , Strike, Volatility, RiskFree, Dividend, Expiry):

    T = Expiry
    r = RiskFree
    q = Dividend
    sigma = Volatility

    x0 = 0.005
    xJ = 3.0

    N = 500
    j1 = 1000
    j2 = 500
    j3 = 200
    J = j1 + j2 + j3
    dt = T / N
    dx1 = (0.9 - x0) / j1
    dx2 = (1.1 - 0.9) / j2
    dx3 = (xJ - 1.1) / j3

    t = np.linspace(0, 1, N + 1)
    x1 = np.linspace(x0, 0.9, j1+1)
    x2 = np.linspace(0.9 + dx2, 1.1, j2)
    x3 = np.linspace(1.1 + dx3, xJ, j3)
    x = np.concatenate([x1, x2, x3])

    # Define option price, option delta, option gamma, and early exercise boundary matrices
    u = np.zeros((N + 1, J + 1))
    v = np.zeros((N + 1, J + 1))
    opgam = np.zeros((N + 1, J + 1))
    eeb = np.ones(N + 1)

    # Set Expiry Boundary Value as Option Payoff
    u[0, :] = np.maximum(0, 1 - x)

    # Set Early Exercise Boundary Condition at Expiry
    if q > 0 :
        eeb[0] = min(1, r / q)
    else:
        eeb[0] = 1

    # Set Expiry Delta and Gamma Conditions
    for j in range(J + 1):
        v[0, j] = -np.heaviside(1 - x[j], 0.5)
        opgam[0, j] = -np.inf if np.isclose(1 - x[j], 0) else 0

    # Define Components of Forward Sweep Equation
    A = np.zeros(J + 1)
    B = np.ones(J + 1)
    conx = np.maximum(np.finfo(float).eps, 0.5 * sigma**2 * x**2)
    D = -(r - q) * x / conx
    f = np.zeros(J + 1)

    # Pre-define Storage Vectors for Riccati Transformation
    R = np.zeros(J + 1)
    w = np.zeros(J + 1)
    phi = np.zeros(J + 1)

    # Define Time Dependent Coefficients with Three Level Backwards Difference for N > 3
    for n in range(1,N+1):
        if n <= 3:
            C = (r + 1 / dt)/ conx
            g = -u[n-1, :]/ (dt * conx)
        else:
            C = (r +3 / (2 * dt)) / conx
            g = (-4 * u[n-1,:] + u[n-2, :]) / (2 * dt* conx)

        # Initialise While-loop Parameters where S denotes Sign-check Variable to Identify Critical Stock Price
        i = J
        phi[J] = x[J] - 1
        S = phi[J]

        while S > 0:
            alpha = 0.5 * C[i-1]
            beta = 1 / (x[i-1] - x[i]) - 0.5 * (A[i-1] - D[i-1])
            gamma = 0.5 * C[i] * (R[i])**2 - (1 / (x[i-1] - x[i]) + 0.5 * (A[i] - D[i])) * R[i] -0.5 * (B[i-1] + B[i])
            R[i-1] = -2 * gamma / (beta - np.sqrt(beta**2 - 4 * alpha * gamma))
            w_num = w[i] + 0.5 * (x[i-1] - x[i]) * ((A[i] - C[i] * R[i]) * w[i] - R[i-1] * g[i-1] - R[i] * g[i] + f[i-1] + f[i])
            w_denom = 1 - 0.5 * (x[i-1] - x[i]) * (A[i-1] - C[i-1] * R[i-1])
            w[i-1] = w_num / w_denom
            phi[i-1] = -1 * R[i-1] + w[i-1] - (1 - x[i-1])
            S = phi[i-1] * phi[i]
            i = i - 1

        # K is Index Of Critical Stock Price
        k = i + 1

        # Compute Phi(k-1) For Polynomial Fitting
        alpha = 0.5 * C[k-1]
        beta = 1 / (x[k-1] - x[k]) - 0.5 * (A[k-1] - D[k-1])
        gamma = 0.5 * C[k] * (R[k])**2 - (1 / (x[k-1] - x[k]) + 0.5 * (A[k] - D[k])) * R[k] - 0.5 * (B[k-1] + B[k])
        R[k-1] = -2 * gamma / (beta - np.sqrt(beta**2 - 4 * alpha * gamma))
        w_num = w[k] + 0.5 * (x[k-1] - x[k]) * ((A[k] - C[k] * R[k]) * w[k] - R[k-1] * g[k-1] - R[k] * g[k] + f[k-1] + f[k])
        w_denom = 1 - 0.5 * (x[k-1] - x[k]) * (A[k-1] - C[k-1] * R[k-1])
        w[k-1] = w_num / w_denom
        phi[k-1] = -1 * R[k-1] + w[k-1] - (1 - x[k-1])

        # Fit Cubic Polynomial Through Critical Stock Price
        x_poly = x[k-1:k+2]
        phi_poly = phi[k-1:k+2]
        poly = np.polyfit(x_poly ,phi_poly ,3)
        dpoly = np.polyder(poly)
        eeb[n] = (phi[k+1] * x[k] - phi[k] * x[k+1]) / (phi[k+1] - phi[k])

        tol = 1.0e-10
        fx = np.polyval(poly,eeb[n])

        # Implementation of Newtons Method for Early Exercise Boundary
        while fx>=tol:
            fx = np.polyval(poly, eeb[n])
            dfx = np.polyval(dpoly, eeb[n])
            eeb[n] = eeb[n] - (fx / dfx)

        # Part 2: Backward Sweep Solving for v(x)

        #Smooth Pasting Condition
        vs = -1

        #Interpolate Values of C, D, g, R, and w at the Early Exercise Boundary
        Cs = np.interp(eeb[n], x[k:k+1], C[k:k+1])
        Ds = np.interp(eeb[n], x[k:k+1], D[k:k+1])
        gs = np.interp(eeb[n], x[k:k+1], g[k:k+1])
        Rs = np.interp(eeb[n], x[k:k+1], R[k:k+1])
        ws = np.interp(eeb[n], x[k:k+1], w[k:k+1])

        #First Step of Backwards Sweep Using Trapezoidal Rule
        vk1_num = vs + 0.5 * (x[k+1] - eeb[n]) * ((Cs * Rs + Ds) * vs + C[k+1] * w[k+1] + Cs * ws + g[k+1] + gs)
        vk1_denom = 1 - 0.5 * (x[k+1] - eeb[n]) * (C[k+1] * R[k+1] + D[k+1])
        v[n,k+1] = vk1_num / vk1_denom
        u[n,k+1] = R[k+1] *v [n,k+1] + w[k+1]

        #Continue Backwards Sweep to Determine v(x) and u(x) for Values Above the Early Exercise Boundary
        for j in range(k+2, J+1):
            v_num = v[n, j-1] + 0.5 * (x[j] - x[j-1]) * ((C[j-1] * R[j-1] + D[j-1]) * v[n, j-1] + C[j] * w[j] + C[j-1] * w[j-1] + g[j] + g[j-1])
            v_denom = 1 - 0.5 * (x[j] - x[j-1]) * (C[j] * R[j] + D[j])
            v[n,j] = v_num / v_denom
            u[n,j] = R[j] * v[n,j] + w[j]
            opgam[n,j] = C[j] * u[n, j] + D[j] * v[n,j] + g[j]

        #Fill in Values Below Early Exercise Boundary Where Exercise Has Occurred
        for j in range(k+1):
            u[n,j] = 1 - x[j]
            v[n,j] = -1
            opgam[n,j] = 0

    option_price = np.interp(Stock, x * Strike, u[-1, :] * Strike)
    delta = np.interp(Stock, x * Strike, v[-1, :])
    gamma = np.interp(Stock, x * Strike, opgam[-1,:])

    print(f"Option Price: {option_price:.6f}\n")
    print(f"Option Delta: {delta:.6f}\n")

    return option_price, eeb, v


In [6]:
price, boundary, delta = MOL_American_Put(36, 40, 0.2, 0.06, 0.00, 1.0)

Option Price: 4.486548

Option Delta: -0.696816

